# 1 | Preparation
Import necessary libraries.

In [1]:
import numpy as np
import pandas as pd

# 2 | Privacy Protection
This dataset contains values that reveals specific individual and institution name. To protect their identity, I used anonymization technique (enumeration) to replace every individual names. As for the specific institution name mentioned, I simply delete them but not to the point that the necessary data is eradicated. As a side note, since this dataset is stored on my public repo, I have censored the individual names beforehand, before it even staged for preparation.

In [ ]:
df = pd.read_csv('data.csv', sep=';')
df['Nama Lengkap'] = [f'Student {i+1}' for i in range(len(df))]
df = df.rename(columns={'Lulus dari [redacted]  ...': 'Lulus dari SMP',
                       'Jurusan ...': 'Jurusan'})
df

,Timestamp,Nama Lengkap,Jenis Kelamin,Lulus dari SMP,Lanjut ke Sekolah Menengah ...,Jurusan
0,8/25/2025 9:45:19,Student 1,Laki-laki,2025,SMK,Agribisnis Tanaman Pangan Dan Hortikultura (ATPH)
1,8/25/2025 9:51:54,Student 2,Laki-laki,2024-2025,SMA,IPS
2,8/25/2025 9:57:09,Student 3,Laki-laki,2025,SMK,Teknik Otomasi Industri
3,8/25/2025 10:05:45,Student 4,Laki-laki,2024-2025,SMK,Teknik kendaraan ringan
4,8/25/2025 10:08:43,Student 5,Perempuan,2025,SMK,USAHA LAYANAN PARIWISATA
...,...,...,...,...,...,...
116,8/26/2025 22:57:26,Student 117,Perempuan,2025,SMK,Teknik Otomasi Industri
117,8/27/2025 21:11:29,Student 118,Perempuan,2024,SMA,ips
118,8/30/2025 9:32:55,Student 119,Perempuan,2025,SMA,IPS
119,8/30/2025 9:42:34,Student 120,Perempuan,2025,SMA,IPS


# 3 | Checking the Data
In this part, I check the data whether it contains any missing values, duplicates, outliers, and inconsistent data points.

## 3.1 Column Overview
All columns seem to be identified as "object". It would be more appropriate if they are converted to "category". However, this change will take place later after all data has been normalized.

In [5]:
print(df.dtypes)

Timestamp                         object
Nama Lengkap                      object
Jenis Kelamin                     object
Lulus dari SMP                    object
Lanjut ke Sekolah Menengah ...    object
Jurusan                           object
dtype: object


## 3.2 Checking for missing values
Status: **OK**

In [7]:
df.isna().sum(axis=0)

Timestamp                         0
Nama Lengkap                      0
Jenis Kelamin                     0
Lulus dari SMP                    0
Lanjut ke Sekolah Menengah ...    0
Jurusan                           0
dtype: int64

## 3.3 Checking for Duplicates
Checking for duplicate data is based on the primary key only. Thus, I check for duplicate on the col 'Nama Lengkap'.\
Status: **OK**

In [9]:
df['Nama Lengkap'].duplicated().sum()

0

## 3.4 Checking for outliers
This stage checks 'Jenis Kelamin', 'Lulus dari SMP', 'Lanjut ke Sekolah Menengah ...', and 'Jurusan'.

In [11]:
df['Jenis Kelamin'].value_counts()

Jenis Kelamin
Perempuan    72
Laki-laki    49
Name: count, dtype: int64

Jenis Kelamin: **OK**

In [13]:
df['Lulus dari SMP'].value_counts()

Lulus dari SMP
2025            55
2024            46
2023             7
2024-2025        4
2022             2
Tahun 2025       1
2023/2024        1
12 Juni 2025     1
thn 2025         1
cth:2024         1
tahun 2024       1
20223            1
Name: count, dtype: int64

In this column, data must be only in one year. Other kinds of data must be converted to one year format. For instance, '2023/2024' must be changed to '2024'.\
In this case, it is more efficient to create a dictionary and change the values accordingly.

In [15]:
normalize_Lulus_dari_SMP = {'2024-2025': '2025',
                           'Tahun 2025': '2025',
                           '2023/2024': '2024',
                           '12 Juni 2025': '2025',
                           'thn 2025': '2025',
                           'cth:2024': '2024',
                           'tahun 2024': '2024',
                           '20223': '2023'}
df['Lulus dari SMP'].map(normalize_Lulus_dari_SMP).fillna(df['Lulus dari SMP']).value_counts() # checks for successful operation and prevents data loss
df['Lulus dari SMP'] = df['Lulus dari SMP'].map(normalize_Lulus_dari_SMP).fillna(df['Lulus dari SMP'])
df['Lulus dari SMP'].value_counts()

Lulus dari SMP
2025    62
2024    49
2023     8
2022     2
Name: count, dtype: int64

Lulus dari SMP: **OK**

In [17]:
df['Lanjut ke Sekolah Menengah ...'].value_counts()

Lanjut ke Sekolah Menengah ...
SMK                 76
SMA                 44
SMAN 1 JALAKSANA     1
Name: count, dtype: int64

Seems like there is one outlier in col 'Lanjut ke Sekolah Menengah ...'.\
I located the specific outlier first, then change it to SMA.

In [19]:
print(df[df['Lanjut ke Sekolah Menengah ...'] == 'SMAN 1 JALAKSANA'])

             Timestamp Nama Lengkap Jenis Kelamin Lulus dari SMP  \
31  8/25/2025 13:15:29   Student 32     Laki-laki           2024   

   Lanjut ke Sekolah Menengah ... Jurusan  
31               SMAN 1 JALAKSANA     IPA  


In [23]:
df.loc[df['Lanjut ke Sekolah Menengah ...'] == 'SMAN 1 JALAKSANA', 'Lanjut ke Sekolah Menengah ...'] = 'SMA' # checkpoint
df['Lanjut ke Sekolah Menengah ...'].value_counts()

Lanjut ke Sekolah Menengah ...
SMK    76
SMA    45
Name: count, dtype: int64

Lanjut ke Sekolah Menengah ...: **OK**

In [32]:
df['Jurusan'].unique()

array(['Agribisnis Tanaman Pangan Dan Hortikultura (ATPH)', 'IPS',
       'Teknik Otomasi Industri', 'Teknik kendaraan ringan',
       'USAHA LAYANAN PARIWISATA', 'AKUNTANSI',
       'Pengembangan Perangkat Lunak dan Gim',
       'desain permodelan dan informasi bangunan',
       'Desain Pemodelan dan Informasi Bangunan', 'Teknik Audio Video',
       'ips', 'Teknik Komputer Jaringan', 'IPA',
       'Desain Komunikasi Visual',
       'Teknik Ketenagalistrikan/Teknik instalasi Tenaga Listrik (TITL)',
       'TSM (TEKNIK SEPEDA MOTOR)', 'Teknik Kontruksi Dan Perumahan',
       'TKP (tehnik konstruksi dan perumahan)',
       'Akuntansi keuangan dan lembaga', 'teknik otomotif',
       'tkj (teknik komputer dan jaringan)',
       'SMK: Teknik kendaraan Ringan', 'teknik audio video',
       'jurusan TSM', 'jurusan teknik permesinan',
       'JURUSAN PEMESINAN (TPM)', 'Teknik Audio Vidio',
       'Teknik Sepeda Motor', 'teknik permesinan smk karya nasional',
       'TEKNIK KENDARAAN RINGAN', '

This column seems to be taking a like to *diversity*. The number of inconsistencies is staggering.

## Numerical Outlier

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Timestamp                       121 non-null    object
 1   Nama Lengkap                    121 non-null    object
 2   Jenis Kelamin                   121 non-null    object
 3   Lulus dari SPENSANA  ...        121 non-null    object
 4   Lanjut ke Sekolah Menengah ...  121 non-null    object
 5   Jurusan ...                     121 non-null    object
dtypes: object(6)
memory usage: 5.8+ KB


In [20]:
for col in df.columns:
    df[col] = df[col].astype('category')

print(df.dtypes)

Timestamp                         category
Nama Lengkap                      category
Jenis Kelamin                     category
Lulus dari SMP                    category
Lanjut ke Sekolah Menengah ...    category
Jurusan                           category
dtype: object
